In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def solve_basis_pursuit(A, b, maxit=100000, tol=1e-03):
    
    M,N = A.shape
    
    tau = 1/np.sum(np.abs(A), axis=0)
    sigma = 1/np.sum(np.abs(A), axis=1)
    
    y = np.zeros(N)
    z = np.zeros(M)

    for it in range(maxit):
        y_ = y.copy()
        y = y - tau*(A.T@z)
        y = np.maximum(0, np.abs(y) - tau)*np.sign(y)
        y_ = 2*y-y_
    
        z = z + sigma*(A@y_-b)
    
        if np.linalg.norm(A@y-b) < tol:
            break

    print("it = ", it, ", feasibility = ", np.linalg.norm(A@y-b))
    return y

In [ ]:
np.random.seed(100)

# length of the signal
n = 100

# sparsity
s = 10

# generate sparse vector
x = np.zeros(n)
idx = np.arange(0,n,1)
r = np.random.choice(idx, s, replace=False)
x[r[:s//2]] = +1
x[r[s//2:]] = -1

#plt.figure(figsize=(8,3))
#plt.stem(x, use_line_collection=True)

# Generate measurement matrix
A_full = np.random.rand(n,n)

In [ ]:
# number of measurements
m = 30

# generate measurement matrix
A = A_full[:m,:]

# make measurement vector
b = A@x

# compute L2 solution using the pseudo inverse
y_l2 = np.linalg.pinv(A)@b

# compute L1 solution using l1 minimization
y_l1 = solve_basis_pursuit(A,b)

# PLot the results
fig=plt.figure(figsize=(10,5))
plt.subplot(311), plt.stem(x, use_line_collection=True, label=r"$x^*$")
plt.legend()
plt.subplot(312), plt.stem(y_l2, use_line_collection=True, label=r"$\ell_2$")
plt.legend()
plt.subplot(313), plt.stem(y_l1, use_line_collection=True, label =r"$\ell_1$")
plt.legend()
fig.savefig("l1_l2.pdf", bbox_inches='tight')